In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [13]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import joblib

import matplotlib.pyplot as plt
from sklearn import preprocessing
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import lightgbm as lgb
#import gpboost as gpb
from lightgbm import LGBMRegressor
import holidays
us_holidays = holidays.country_holidays('US')
from sklearn.model_selection import RandomizedSearchCV

# Training loop
import torch
import torch.nn as nn
import torch.optim as optim

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"



In [4]:
df = pd.read_csv(f'{data_dir}/final_model_input_partial_scale_4.csv')

df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])

df = df.sort_values(by=['start_station_cluster', 'started_at_hourly'])
df_test = df[pd.to_datetime(
    df['started_at_hourly']) >= pd.to_datetime("2024-01-01 00:00:00")]
df_train = df[pd.to_datetime(
    df['started_at_hourly']) < pd.to_datetime("2024-01-01 00:00:00")]

del df

In [5]:
execfile('/content/drive/MyDrive/Thesis/models_training/model_training.py')

Using device: cpu


In [6]:
target_scaler = joblib.load(f'{models_dir}/target_scaler.sav')

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Normal Model

In [7]:
@scope.define
def to_int(x):
    """
    Custom Hyperopt scope function to cast a value to an integer.

    :param x: Value to be cast to an integer.
    :type x: Any
    :return: Integer representation of the input value.
    :rtype: int
    """
    return int(x)


In [8]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            #'flag_added',
            'is_holiday',
            "total_demand_1h",
            "demand_degrees_1h",
            ]

## XBGoost

In [9]:


model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='xgb', tuning=False)

overall
MSE: 4.118907865438012
RMSE: 2.029509267147606
MAE: 1.0621890473894937
MAPE: 0.42403875306465777

Non-zero
MSE: 8.390017436802882
RMSE: 2.8965526815169236
MAE: 1.791577687167561
MAPE: 0.372941022191347

Zeros
MSE: 0.6267138534075706
RMSE: 0.7916526090448831
MAE: 0.46581787083313386
MAPE: 0.46581787083313386


In [ ]:
#11-01
3.532484120211517
1.8794903884328638
0.3918793693597117

#11-09 scaled
4.118907865438012
2.029509267147606
0.4240387530646576

#11-28
4.164198927390364
2.040636892587793
0.4290005681691596

0.4290005681691596

In [ ]:
suf = "_a_xgb_11_24"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

filename = f'{models_dir}/test_predictions_list{suf}.sav'
joblib.dump(preds_list, filename)
del preds_list

## LightGBM

In [10]:

model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='lgbm')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.048582 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 678
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 11
[LightGBM] [Info] Start training from score 1.005143
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049502 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 678
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 11
[LightGBM] [Info] Start training from score 1.005143
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047777 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not eno

In [ ]:
# MSE: 4.672158282299454
# RMSE: 2.1615175877839747
# MAE: 1.182334791667978
# MAPE: 0.5313402888419341

# Non-zero
# MSE: 9.167820398571836
# RMSE: 3.027840880656022
# MAE: 1.798114663167833
# MAPE: 0.3509247478453334

# Zeros
# MSE: 0.9963630210831674
# RMSE: 0.9981798540759914
# MAE: 0.6788537303536266
# MAPE: 0.6788537303536266

In [ ]:
#11_15
4.640340563964935
2.154144972829112
0.5311516974241975

#11-28
4.672158282299454
2.1615175877839747
0.5313402888419341

In [ ]:
suf = "_a_lgbm_11_15"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

filename = f'{models_dir}/test_predictions_list{suf}.sav'
joblib.dump(preds_list, filename)
del preds_list

In [ ]:
output = torch.rand(76, 10)

In [ ]:
output.shape

torch.Size([76, 10])

In [ ]:
output[0:2]

tensor([[0.2664, 0.9718, 0.9941, 0.1413, 0.4667, 0.0349, 0.0522, 0.8430, 0.6883,
         0.3038],
        [0.9788, 0.0647, 0.4659, 0.7147, 0.7517, 0.6183, 0.7326, 0.7998, 0.7791,
         0.7308]])

## Random Forest

In [11]:


model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='rf')

KeyboardInterrupt: 

In [ ]:
########
# overall
# MSE: 3.9894494268102862
# RMSE: 1.997360615114428
# MAE: 1.0467235369137318
# MAPE: 0.42419015833572665

# Non-zero
# MSE: 7.849115898691792
# RMSE: 2.8016273661377227
# MAE: 1.7584333629970268
# MAPE: 0.37451380066782775

# Zeros
# MSE: 0.8336642809770844
# RMSE: 0.9130521786716707
# MAE: 0.464807116502848
# MAPE: 0.464807116502848

In [ ]:
5.636537048223255
2.374139222586421
0.33886059404561014

#11-01
3.566440816395711
1.8885022680409231
0.3805276715588524

#11-28
3.9729415562562256
1.9932239102158658
0.4240974842074029

In [ ]:
suf = "_a_rf_11_28"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

filename = f'{models_dir}/test_predictions_list{suf}.sav'
joblib.dump(preds_list, filename)
del preds_list

NameError: name 'le' is not defined

## LSTM

In [ ]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            #'flag_added',
            'is_holiday',
            "total_demand_1h",
            "demand_degrees_1h",
            ]

model, model_preds = train_model(df_train, df_test, FEATURES, model='lstm', epochs=20, batch_size=32)

Epoch [1/20], Loss: 0.0006


KeyboardInterrupt: 

In [ ]:
model

In [ ]:
suf = "_a_lstm_11_16"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model


0.38054293838781955

# Results

### 1-11


In [ ]:
#b XGB (Weather)
# overall
# MSE: 4.061524407266129
# RMSE: 2.015322407771553
# MAE: 1.067188920933303
# MAPE: 0.4350008358415209
# Non-zero
# MSE: 8.207877787744946
# RMSE: 2.864939403852191
# MAE: 1.7725544429383784
# MAPE: 0.36717203592849273
# Zeros
# MSE: 0.6713349950935068
# RMSE: 0.8193503494192864
# MAE: 0.4904598038079627
# MAPE: 0.4904598038079627

#d XGB (Weather + lag)
# overall
# MSE: 3.680589537470945
# RMSE: 1.9184862619969278
# MAE: 1.0251868492937535
# MAPE: 0.41571022142206315
# Non-zero
# MSE: 7.468849180206185
# RMSE: 2.732919534162355
# MAE: 1.7112843503707298
# MAPE: 0.35639052922218767
# Zeros
# MSE: 0.5831888692532394
# RMSE: 0.7636680360295561
# MAE: 0.4642118743144978
# MAPE: 0.4642118743144978

#i RF (Embedding)
# overall
# MSE: 4.157300202907574
# RMSE: 2.0389458558057822
# MAE: 1.0809562898520895
# MAPE: 0.4420209656932694
# Non-zero
# MSE: 8.42366584368381
# RMSE: 2.902355223552729
# MAE: 1.7920848122688264
# MAPE: 0.37170299083731634
# Zeros
# MSE: 0.6689849782784607
# RMSE: 0.8179150189833053
# MAE: 0.499515161669046
# MAPE: 0.499515161669046

#l XGB (var)
# overall
# MSE: 3.5321116400032615
# RMSE: 1.8793912950748872
# MAE: 1.000908126683838
# MAPE: 0.3998485691043102
# Non-zero
# MSE: 7.188246408303783
# RMSE: 2.681090525943461
# MAE: 1.692612846449564
# MAPE: 0.35643054927011647
# Zeros
# MSE: 0.5427404399206347
# RMSE: 0.736709196305187
# MAE: 0.4353485127011067
# MAPE: 0.4353485127011067

#n XGB (mean)
# overall
# MSE: 3.5222329685487885
# RMSE: 1.876761297701119
# MAE: 0.9917181586341258
# MAPE: 0.39208096407077486
# Non-zero
# MSE: 7.195642903461277
# RMSE: 2.6824695531284743
# MAE: 1.6860098611609182
# MAPE: 0.3529895406249029
# Zeros
# MSE: 0.5187370469175498
# RMSE: 0.720234022327153
# MAE: 0.42404334588244
# MAPE: 0.42404334588244

#m XGB (PCA)
# overall
# MSE: 3.5229701168797694
# RMSE: 1.876957675836024
# MAE: 0.9956896304822397
# MAPE: 0.39470800631747033
# Non-zero
# MSE: 7.185092966319084
# RMSE: 2.6805023720040024
# MAE: 1.6903708260908172
# MAPE: 0.35436177838032124
# Zeros
# MSE: 0.5287028726317953
# RMSE: 0.7271195724444469
# MAE: 0.42769635589172467
# MAPE: 0.42769635589172467

#o XGB (CV-PCA)
# MSE: 3.4781439953315743
# RMSE: 1.864978282804273
# MAE: 0.9930974108872644
# MAPE: 0.39507594954338343
# Non-zero
# MSE: 7.081684897418646
# RMSE: 2.6611435319085377
# MAE: 1.6827883969567792
# MAPE: 0.3533599236804838
# Zeros
# MSE: 0.5317752005001515
# RMSE: 0.7292291824249435
# MAE: 0.42918428914226003
# MAPE: 0.42918428914226003

### 1-03



In [ ]:
#a RF
#overall
#MSE: 3.9894494268102862
#RMSE: 1.997360615114428
#MAE: 1.0467235369137318
#MAPE: 0.42419015833572665
#Non-zero
#MSE: 7.849115898691792
#RMSE: 2.8016273661377227
#MAE: 1.7584333629970268
#MAPE: 0.37451380066782775
#Zeros
#MSE: 0.8336642809770844
#RMSE: 0.9130521786716707
#MAE: 0.464807116502848
#MAPE: 0.464807116502848

#b RF
#overall
#MSE: 3.7578975499504215
#RMSE: 1.9385297392483876
#MAE: 1.0215060223891448
#MAPE: 0.4136491770939839
#Non-zero
#MSE: 7.437731335102529
#RMSE: 2.727220441237292
#MAE: 1.7161809433055792
#MAPE: 0.3648879713187312
#Zeros
#MSE: 0.7491492855810391
#RMSE: 0.8655341042275799
#MAE: 0.45351787818499956
#MAPE: 0.45351787818499956

#c XGB
# overall
# MSE: 3.6956497497043204
# RMSE: 1.922407279871859
# MAE: 1.0243547987928214
# MAPE: 0.41613950241680286
# Non-zero
# MSE: 7.497623429192625
# RMSE: 2.738178852666974
# MAE: 1.7106544144195146
# MAPE: 0.3585645896372869
# Zeros
# MSE: 0.5870360521251313
# RMSE: 0.7661827798411626
# MAE: 0.4632145685775499
# MAPE: 0.4632145685775499

#d XGB
# MSE: 3.694719359705295
# RMSE: 1.9221652789771475
# MAE: 1.026340039350822
# MAPE: 0.41533332978086984
# Non-zero
# MSE: 7.494243214728542
# RMSE: 2.7375615453772983
# MAE: 1.7178353150823946
# MAPE: 0.35954005383914883
# Zeros
# MSE: 0.5881087159425573
# RMSE: 0.766882465533381
# MAE: 0.4609516734219524
# MAPE: 0.4609516734219524


#e XGB
# overall
# MSE: 3.647029207344911
# RMSE: 1.9097196672142513
# MAE: 1.024807643990257
# MAPE: 0.41650298269082386
# Non-zero
# MSE: 7.392688142556299
# RMSE: 2.71894982347161
# MAE: 1.7099444317900678
# MAPE: 0.3576559447835239
# Zeros
# MSE: 0.5844602223788978
# RMSE: 0.7644999819351848
# MAE: 0.4646181785133301
# MAPE: 0.4646181785133301

#f XGB
# MSE: 3.6794573989792814
# RMSE: 1.9181911789441848
# MAE: 1.0264590525877988
# MAPE: 0.4178085847938585
# Non-zero
# MSE: 7.460762608077842
# RMSE: 2.7314396585093808
# MAE: 1.7112004481930954
# MAPE: 0.3581432178725427
# Zeros
# MSE: 0.5877428952341267
# RMSE: 0.7666439168441413
# MAE: 0.46659287225048623
# MAPE: 0.46659287225048623


#g XGB
# overall
# MSE: 3.660790538947053
# RMSE: 1.913319246479022
# MAE: 1.0307203251880628
# MAPE: 0.4214031077956738
# Non-zero
# MSE: 7.3958121340257295
# RMSE: 2.7195242477363077
# MAE: 1.7147152117829707
# MAPE: 0.36017576726440015
# Zeros
# MSE: 0.6069189790633743
# RMSE: 0.7790500491389333
# MAE: 0.47146451418275814
# MAPE: 0.47146451418275814

#h XGB
# overall
# MSE: 3.6996683466347555
# RMSE: 1.9234521950479444
# MAE: 1.0292779393860483
# MAPE: 0.41762477535448683
# Non-zero
# MSE: 7.501210592355168
# RMSE: 2.7388338015212184
# MAE: 1.7193072839206256
# MAPE: 0.3595749254768739
# Zeros
# MSE: 0.591407402923303
# RMSE: 0.7690301703595919
# MAE: 0.46508816507901807
# MAPE: 0.46508816507901807


#i RF
#overall
#MSE: 3.7223187275046272
#RMSE: 1.9293311606628416
#MAE: 1.0256152756128958
#MAPE: 0.41466506537131165
#Non-zero
#MSE: 7.505198277667845
#RMSE: 2.739561694444541
#MAE: 1.724459556304582
#MAPE: 0.36628989559694997
#Zeros
#MSE: 0.6293169928076513
#RMSE: 0.7932950225531806
#MAE: 0.4542181312419671
#MAPE: 0.4542181312419671

#j XGB
# overall
# MSE: 3.534265459201861
# RMSE: 1.8799642175323075
# MAE: 0.9973491322851492
# MAPE: 0.3947481051421412
# Non-zero
# MSE: 7.20667522373176
# RMSE: 2.684525139336892
# MAE: 1.6938633612235703
# MAPE: 0.3542543082153698
# Zeros
# MSE: 0.5315873084478928
# RMSE: 0.7291003418240132
# MAE: 0.4278571117806337
# MAPE: 0.4278571117806337

#k XGB
# overall
# MSE: 3.5493534223843457
# RMSE: 1.8839727764445924
# MAE: 1.0002914774284006
# MAPE: 0.39750226024213126
# Non-zero
# MSE: 7.2277160864076855
# RMSE: 2.6884412001023352
# MAE: 1.695786534866335
# MAPE: 0.3557591269694381
# Zeros
# MSE: 0.5418079930888501
# RMSE: 0.7360760783294414
# MAE: 0.4316327637149571
# MAPE: 0.4316327637149571

#l XGB
# overall
# MSE: 3.5321116400032615
# RMSE: 1.8793912950748872
# MAE: 1.000908126683838
# MAPE: 0.3998485691043102
# Non-zero
# MSE: 7.188246408303783
# RMSE: 2.681090525943461
# MAE: 1.692612846449564
# MAPE: 0.35643054927011647
# Zeros
# MSE: 0.5427404399206347
# RMSE: 0.736709196305187
# MAE: 0.4353485127011067
# MAPE: 0.4353485127011067

#m XGB
# overall
# MSE: 3.516988135177505
# RMSE: 1.875363467485038
# MAE: 0.9951089718690401
# MAPE: 0.3956547845817011
# Non-zero
# MSE: 7.171410815720373
# RMSE: 2.6779489942342765
# MAE: 1.6863347955596644
# MAPE: 0.3537213083887933
# Zeros
# MSE: 0.5290167920911575
# RMSE: 0.7273354054981495
# MAE: 0.4299409184346657
# MAPE: 0.4299409184346657

#n XGB
# overall
# MSE: 3.5330230608451085
# RMSE: 1.8796337571040558
# MAE: 0.993714406990086
# MAPE: 0.39265990380160887
# Non-zero
# MSE: 7.216614310539205
# RMSE: 2.6863756830605814
# MAE: 1.6903601604710088
# MAPE: 0.3541890994290781
# Zeros
# MSE: 0.521202574853991
# RMSE: 0.7219436091925677
# MAE: 0.4241148478677746
# MAPE: 0.4241148478677746

#o XGB
# overall
# MSE: 3.4851078325990033
# RMSE: 1.8668443514655964
# MAE: 0.9975963067263892
# MAPE: 0.39848029011836106
# Non-zero
# MSE: 7.092221388506636
# RMSE: 2.6631224884534763
# MAE: 1.6868747427704824
# MAPE: 0.35501302415469377
# Zeros
# MSE: 0.5358179232256244
# RMSE: 0.7319958491860622
# MAE: 0.4340204989051717
# MAPE: 0.4340204989051717


### 11-28 Results

In [ ]:
#a RF
3.9729415562562256
1.9932239102158658
0.4240974842074029

#b RF
3.761551942397637
1.9394720782722388
0.41353221812323404

#c XGB
3.6956497497043204
1.922407279871859
0.41613950241680286

#d XGB
3.694719359705295
1.9221652789771475
0.41533332978086984

#e XGB
3.647029207344911
1.9097196672142513
0.41650298269082386

#f XGB
3.6794573989792814
1.9181911789441848
0.4178085847938585

#g XGB
3.660790538947053
1.913319246479022
0.4214031077956738

#i RF
3.723580769803055
1.9296582002528466
0.41509447338288163

#j XGB
3.534265459201861
1.8799642175323075
0.3947481051421412

#h XGB
3.6996683466347555
1.9234521950479444
0.41762477535448683

#k XGB
3.5493534223843457
1.8839727764445924
0.39750226024213126

#l XGB
3.5321116400032615
1.8793912950748872
0.3998485691043102

#m XGB
3.516988135177505
1.875363467485038
0.3956547845817011

#n XGB
3.5330230608451085
1.8796337571040558
0.39265990380160887

#o XGB
3.4851078325990033
1.8668443514655964
0.39848029011836106

### Current Results

In [ ]:
#a
4.118907865438012
2.029509267147606
0.4240387530646576

#b
4.061524407266127
2.0153224077715524
0.4350008358415209

#c
3.6841001457201483
1.9194009861725476
0.4132425073915699

#d
3.668979611469117
1.9154580683139781
0.4155907549079753

#e
3.637830331305797
1.9073097103789403
0.4170961189192818

#f
3.6949065265972303
1.9222139648325391
0.4156726768881891

#g
3.649102210215345
1.910262340678721
0.4127466890916304

#h
3.6863289253860314
1.9199814908967303
0.419749131302065

#i
4.112374062928123
2.0278989281835824
0.43646158777503297

#j
3.536247192876756
1.880491210528982
0.39338313284835247

#k
3.5103431029572603
1.8735909646871327
0.3949223304344264

#l
3.526444658025326
1.877883025650247
0.39191876965741634

#m
3.511820258766204
1.873985127680101
0.3926758989622167

#n
3.5379750563532233
1.8809505725438995
0.3987008225026617

#o
3.519954687632939
1.8761542281041126
0.39561652953215365

### Outdated Results

In [ ]:
#e
3.189635202926419
1.7859549834546276
0.37270374298234266

#f
3.1785156823909997
1.7828392194449278
0.374173850198251

#g
3.1979816352695654
1.788290142921323
0.3706682703327132

#h
3.201650782753264
1.789315730315157
0.37396526641032307

#i
3.581524793531
1.8924916891577093
0.3959399998524082

#k
3.063089630068344
1.7501684576258205
0.35523741143766596